#	Тестовое задание №1
Сервис Яндекс.Еда привлекает новых клиентов в том числе и с помощью скидок. При этом для повышения экономической эффективности необходимо отслеживать основные метрики по скидкам.

**Напишите два SQL запроса, которые отвечают на следующие вопросы:**

**1.Какова доля заказов с промокодами?**

**2.Какой самый популярный промокод (название) и какая сумма общих затрат по нему? Самый популярный промокод – это промокод, по которому получились наибольшие затраты в деньгах.**

Даны две таблицы в Базе данных:

1.orders (id, promocode_id)

2.promocodes (id, name, discount)

name – название промокода
discount – скидка в рублях
*Один промокод может применяться в разных заказах, но в одном заказе может быть применено не более одного промокода.

# Импорт библиотек и создание БД

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
conn = sqlite3.connect('mydata.db')
cur=conn.cursor()

In [3]:
def select(sql):
  return pd.read_sql(sql, conn)

In [ ]:
orders = pd.DataFrame({'id':[1,2,3,4,5,6], 'promocode_id': [0.0, 1.0, 1.0, 2.0, 0.0, 3.0]})

In [ ]:
orders

,id,promocode_id
0,1,0.0
1,2,1.0
2,3,1.0
3,4,2.0
4,5,0.0
5,6,3.0


In [ ]:
orders.to_sql('orders',conn, index=False,if_exists='replace')

In [ ]:
promocodes = pd.DataFrame({'promocode_id':[1, 2, 3], 'name': ['eda', 'lavka', 'big'], 'discount': [300, 500, 800]})

In [ ]:
promocodes

,promocode_id,name,discount
0,1,eda,300
1,2,lavka,500
2,3,big,800


In [ ]:
promocodes.to_sql('promo',conn, index=False,if_exists='replace')

# Задание 1

1) Процент заказов без промокодов и с промокодами с помощью pandas

In [ ]:
df = pd.merge(orders, promocodes, how='outer', on='promocode_id')

In [ ]:
df

,id,promocode_id,name,discount
0,1,0.0,NaN,NaN
1,5,0.0,NaN,NaN
2,2,1.0,eda,300.0
3,3,1.0,eda,300.0
4,4,2.0,lavka,500.0
5,6,3.0,big,800.0


In [ ]:
df['has_discount'] = (df['promocode_id'] > 0)*1

In [ ]:
df.groupby('has_discount').size() / len(df) 

has_discount
0    0.333333
1    0.666667
dtype: float64

2) Процент заказов без промокодов с помощью sql-запроса

In [ ]:
sql = ''' select avg(case when promocode_id =0 then 1 else 0 end) as pers_without_promo
from orders
'''

In [ ]:
select (sql)

,pers_without_promo
0,0.333333


# Задание 2

1) Сумма затрат по популярному промокоду средствами pandas

In [ ]:
df.pivot_table(index='name', values='discount', aggfunc='sum')

,discount
name,
big,800.0
eda,600.0
lavka,500.0


2) Сумма затрат по популярному промокоду c помощью запроса

In [ ]:
sql = ''' 
SELECT p.name, sum(p.discount) as expense
FROM orders o
INNER JOIN
promo p on p.promocode_id = o.promocode_id
GROUP BY p.name
ORDER BY sum(p.discount) desc

'''

In [ ]:
select(sql)

,name,expense
0,big,800
1,eda,600
2,lavka,500


In [ ]:
sql =''' SELECT t.name, max(t.expense) as max_expense
FROM

(SELECT p.name, sum(p.discount) as expense
FROM orders o
INNER JOIN
promo p on p.promocode_id = o.promocode_id
GROUP BY p.name) t 

'''


In [ ]:
select (sql)

,name,max(t.expense)
0,big,800
